# Úvod do pravdepodobnosti a štatistiky
V tomto zošite si zahráme s niektorými konceptmi, o ktorých sme už predtým hovorili. Mnohé koncepty z pravdepodobnosti a štatistiky sú dobre zastúpené v hlavných knižniciach pre spracovanie dát v Pythone, ako sú `numpy` a `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Náhodné premenné a rozdelenia
Začnime výberom vzorky 30 hodnôt z rovnomerného rozdelenia od 0 do 9. Vypočítame tiež priemer a rozptyl.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Na vizuálne odhadnutie, koľko rôznych hodnôt je v vzorke, môžeme nakresliť **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Analyzovanie reálnych dát

Pri analyzovaní dát zo skutočného sveta sú stredná hodnota a rozptyl veľmi dôležité. Načítať si dáta o hráčoch baseballu zo stránky [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Používame tu balík [**Pandas**](https://pandas.pydata.org/) na analýzu dát. O Pandas a práci s dátami v Pythone budeme hovoriť neskôr v tomto kurze.

Vypočítajme priemerné hodnoty pre vek, výšku a váhu:


In [ ]:
df[['Age','Height','Weight']].mean()

Teraz sa zamerajme na výšku a vypočítajme štandardnú odchýlku a rozptyl:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Okrem priemeru má zmysel pozrieť sa aj na mediánovú hodnotu a kvartily. Dá sa to vizualizovať pomocou **box plotu**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Môžeme tiež vytvoriť krabicové grafy pre podmnožiny našich dát, napríklad zoskupené podľa úlohy hráča.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Poznámka**: Tento diagram naznačuje, že priemerne sú výšky prvých bazemanov väčšie než výšky druhých bazemanov. Neskôr sa naučíme, ako túto hypotézu formálnejšie otestovať a ako ukázať, že naše dáta sú štatisticky významné.  

Vek, výška a váha sú všetko spojité náhodné premenné. Ako si myslíte, aké je ich rozdelenie? Dobrým spôsobom, ako to zistiť, je nakresliť histogram hodnôt: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normálne rozdelenie

Vytvorme umelý vzorku hmotností, ktorá nasleduje normálne rozdelenie so rovnakým priemerom a rozptylom ako naše skutočné údaje:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Keďže väčšina hodnôt v reálnom živote je normálne rozdelená, nemali by sme používať generátor rovnomerných náhodných čísel na generovanie vzorových dát. Tu je, čo sa stane, ak sa pokúsime generovať váhy s rovnomerným rozdelením (generované pomocou `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervaly spoľahlivosti

Teraz vypočítajme intervaly spoľahlivosti pre hmotnosti a výšky hráčov baseballu. Použijeme kód [z tejto diskusie na stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Testovanie hypotéz

Preskúmajme rôzne úlohy v našej databáze baseballových hráčov:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Otestujme hypotézu, že prví basebaloví hráči sú vyšší ako druhí basebaloví hráči. Najjednoduchší spôsob, ako to urobiť, je otestovať intervaly spoľahlivosti:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Vidíme, že intervaly sa neprekrývajú.

Štatisticky správnejším spôsobom, ako dokázať hypotézu, je použitie **Studentovho t-testu**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dve hodnoty vrátené funkciou `ttest_ind` sú:
* p-hodnota sa dá považovať za pravdepodobnosť, že dve rozdelenia majú rovnaký priemer. V našom prípade je veľmi nízka, čo znamená, že existuje silný dôkaz podporujúci, že prví basemanovia sú vyšší.
* t-hodnota je medzihodnota normalizovaného rozdielu priemerov, ktorá sa používa v t-teste a porovnáva sa s prahovou hodnotou pre danú úroveň spoľahlivosti.


## Simulácia normálneho rozdelenia pomocou centrálnej limitnej vety

Pseudo-náhodný generátor v Pythone je navrhnutý tak, aby nám poskytoval rovnomerné rozdelenie. Ak chceme vytvoriť generátor pre normálne rozdelenie, môžeme použiť centrálnu limitnú vetu. Aby sme získali hodnotu s normálnym rozdelením, jednoducho vypočítame priemer vzorky vygenerovanej rovnomerne.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelácia a Evil Baseball Corp

Korelácia nám umožňuje nájsť vzťahy medzi dátovými sekvenciami. V našom hračkárskom príklade si predstavme, že existuje zlá baseballová korporácia, ktorá platí svojim hráčom podľa ich výšky - čím vyšší hráč je, tým viac peňazí dostane. Predpokladajme základný plat 1000 dolárov a dodatočný bonus od 0 do 100 dolárov, závisiac od výšky. Vezmeme skutočných hráčov z MLB a vypočítame ich imaginárne platy:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Teraz vypočítajme kovarianciu a koreláciu týchto sekvencií. `np.cov` nám poskytne takzvanú **kovariančnú maticu**, ktorá je rozšírením kovariancie na viacero premenných. Prvok $M_{ij}$ kovariančnej matice $M$ je korelácia medzi vstupnými premennými $X_i$ a $X_j$, a diagonálne hodnoty $M_{ii}$ sú rozptylom $X_{i}$. Podobne `np.corrcoef` nám poskytne **korelačnú maticu**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelácia rovná 1 znamená, že medzi dvoma premennými existuje silný **lineárny vzťah**. Lineárny vzťah môžeme vizuálne vidieť tak, že vykreslíme jednu hodnotu proti druhej:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Uvidíme, čo sa stane, ak vzťah nie je lineárny. Predpokladajme, že naša korporácia sa rozhodla skryť zrejmú lineárnu závislosť medzi výškami a platmi a do vzorca zaviedla nejakú nelineárnosť, napríklad `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

V tomto prípade je korelácia o niečo menšia, ale stále je dosť vysoká. Teraz, aby sme vzťah urobili ešte menej zjavný, môžeme chcieť pridať nejakú extra náhodnosť pridaním náhodnej premenej k platu. Pozrime sa, čo sa stane:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Viete hádať, prečo bodky vytvárajú takéto vertikálne čiary?

Pozorovali sme koreláciu medzi umelo vytvoreným konceptom, ako je plat, a pozorovanou premennou *výškou*. Pozrime sa tiež, či spolu korelujú dve pozorované premenné, ako je výška a hmotnosť:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Bohužiaľ sme nedostali žiadne výsledky - len niektoré zvláštne hodnoty `nan`. Je to spôsobené tým, že niektoré hodnoty v našej sérii sú nedefinované, reprezentované ako `nan`, čo spôsobuje, že výsledok operácie je tiež nedefinovaný. Pohľadom na maticu vidíme, že stĺpec `Weight` je problémový, pretože bola vypočítaná samokorelácia medzi hodnotami `Height`.

> Tento príklad ukazuje dôležitosť **prípravy dát** a **čistenia**. Bez správnych dát nemôžeme nič vypočítať.

Použime metódu `fillna` na doplnenie chýbajúcich hodnôt a vypočítajme koreláciu:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Skutočne existuje korelácia, ale nie taká silná ako v našom umelom príklade. Ak sa naozaj pozrieme na rozptylový graf jednej hodnoty voči druhej, vzťah by bol oveľa menej zjavný:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Záver

V tomto zápisníku sme sa naučili vykonávať základné operácie s údajmi na výpočet štatistických funkcií. Teraz vieme, ako použiť spoľahlivý aparát matematiky a štatistiky na overenie niektorých hypotéz a ako vypočítať intervaly spoľahlivosti pre ľubovoľné premenné na základe vzorky údajov.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zrieknutie sa zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, majte prosím na pamäti, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Originálny dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
